# Wrappers example

We will see on this notebook what are the wrappers defined by Sinergym and how to use them. You can also implement your own wrappers inheriting from *gym.Wrapper* or some of its variants.

In [4]:
import gymnasium as gym
import numpy as np
import sinergym
from sinergym.utils.wrappers import *
from sinergym.utils.constants import RANGES_5ZONE

## Multi-Objective Wrapper

[MO-Gymnasium](https://github.com/Farama-Foundation/MO-Gymnasium) is an open source Python library for developing and comparing multi-objective reinforcement learning algorithms. These environments return a reward vector instead of a scalar value; one for each objective.

In order to be more general as possible, it could be interesting that Sinergym would give that reward vector too. In this way, Sinergym would have compatibility with both; multi-objective algorithms and algorithms that work with a traditional reward value.

We can transform reward returned in a vector using the next wrapper:

In [5]:
env=gym.make('Eplus-5Zone-hot-discrete-v1')
env=MultiObjectiveReward(env,reward_terms=['reward_energy','reward_comfort'])

[2023-06-28 14:26:04,881] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:04,881] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:04,882] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-06-28 14:26:04,882] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-06-28 14:26:04,883] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating building model OutPut:Variable and variables XML tree model for BVCTB connection.
[2023-06-28 14:26:04,883] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating building model OutPut:Variable and variables XML tree model for BVCTB connection.
[2023-06

You have to ensure that `reward_terms` are available in `info` dict returned in step method of the environment. Otherwise, we will encounter an execution error.
By default, Sinergym environments return in `info` dict all reward terms specified in reward class used, so if the objective exists in reward term you shouldn't have any problem.

In [6]:
env.reset()
action = env.action_space.sample()
obs, reward, terminated, truncated, info = env.step(action)
env.close()
print(reward)

[2023-06-28 14:26:04,890] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-06-28 14:26:04,890] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-06-28 14:26:04,993] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-discrete-v1-res2/Eplus-env-sub_run1
[2023-06-28 14:26:04,993] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-discrete-v1-res2/Eplus-env-sub_run1


/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[2023-06-28 14:26:10,310] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-06-28 14:26:10,310] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[-0.05934675774482191, -0.0]


## Previous Observations Wrappers

This Wrapper will add observation values from previous timestep to current environment observation.
It is possible to select the variables you want to track its previous observation values. 
Observation space will be updated with the new dimension.

In [7]:
env=gym.make('Eplus-5Zone-hot-discrete-v1')
env = PreviousObservationWrapper(env, previous_variables=[
        'Zone Thermostat Heating Setpoint Temperature(SPACE1-1)',
        'Zone Thermostat Cooling Setpoint Temperature(SPACE1-1)',
        'Zone Air Temperature(SPACE1-1)'])

[2023-06-28 14:26:10,375] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:10,375] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:10,375] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:10,377] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-06-28 14:26:10,377] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-06-28 14:26:10,377] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-06-28 14:26:10,3

We can see the observation values has been updated:

In [8]:
env.reset()
obs,_,_,_,_=env.step(env.action_space.sample())
obs_dict=dict(zip(env.variables['observation'],obs))
env.close()
print('NEW OBSERVATION: ',obs_dict)

[2023-06-28 14:26:10,386] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-06-28 14:26:10,386] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-06-28 14:26:10,386] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-06-28 14:26:10,520] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-discrete-v1-res3/Eplus-env-sub_run1
[2023-06-28 14:26:10,520] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-discrete-v1-res3/Eplus-env-sub_run1
[2023-06-28 14:26:10,520] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-discrete-v1-res3/Eplus-env-sub_run1


/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[2023-06-28 14:26:15,810] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-06-28 14:26:15,810] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-06-28 14:26:15,810] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
NEW OBSERVATION:  {'year': 1991.0, 'month': 1.0, 'day': 1.0, 'hour': 0.0, 'Site Outdoor Air Drybulb Temperature(Environment)': 4.4, 'Site Outdoor Air Relative Humidity(Environment)': 65.0, 'Site Wind Speed(Environment)': 3.875, 'Site Wind Direction(Environment)': 145.0, 'Site Diffuse Solar Radiation Rate per Area(Environment)': 0.0, 'Site Direct Solar Radiation Rate per Area(Environment)': 0.0, 'Zone Thermostat Heating Setpoint Temperature(SPACE1-1)': 17.0, 'Zone Thermostat Cooling Setpoint Temperature(SPACE1-1)': 28.0, 'Zone Air Temperature(SPACE1-1)': 19.598848, 'Zone Air Relative Humidity(SPACE1-1)': 25.017408, 'Zone Peo

## Datetime Wrapper

This wrapper will substitute `day` value by `is_weekend` flag, and `hour` and `month` by sin and cos values. 
Observation space is also updated automatically.

In [9]:
env=gym.make('Eplus-5Zone-hot-discrete-v1')
env = DatetimeWrapper(env)

[2023-06-28 14:26:15,872] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:15,872] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:15,872] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:15,872] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:15,873] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-06-28 14:26:15,873] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-06-28 14:26:15,873] EPLUS_ENV_5Zone-ho

Concretely, this wrapper deletes the observation variables `month`, `day` and `hour`, and adds `month_sin`, `month_cos`, `is_weekend`, `hour_sin` and `hour_cos` instead:

In [10]:
env.reset()
obs,_,_,_,_=env.step(env.action_space.sample())
obs_dict=dict(zip(env.variables['observation'],obs))
env.close()
print('NEW OBSERVATION: ',obs_dict)

[2023-06-28 14:26:15,886] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-06-28 14:26:15,886] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-06-28 14:26:15,886] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-06-28 14:26:15,886] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-06-28 14:26:15,990] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-discrete-v1-res4/Eplus-env-sub_run1
[2023-06-28 14:26:15,990] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-discrete-v1-res4/Eplus-env-sub_run1
[2023-06-28 14:26:15,990] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working di

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[2023-06-28 14:26:21,790] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-06-28 14:26:21,790] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-06-28 14:26:21,790] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-06-28 14:26:21,790] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
NEW OBSERVATION:  {'year': 1991.0, 'month_cos': 1.0, 'month_sin': 0.0, 'is_weekend': 0.0, 'hour_cos': 1.0, 'hour_sin': 0.0, 'Site Outdoor Air Drybulb Temperature(Environment)': 4.400000095367432, 'Site Outdoor Air Relative Humidity(Environment)': 65.0, 'Site Wind Speed(Environment)': 3.875, 'Site Wind Direction(Environment)': 145.0, 'Site Diffuse Solar Radiation Rate per Area(Environment)': 0.0, 'Site Direct Solar Radiation Rate per Area(Environment)': 0.0, 'Zone Thermostat Heating Setpoint Temperature(

## Discrete Incremental Wrapper

A wrapper for an incremental setpoint action space environment. This wrapper
  will update a *continuous* environment, converting it in a *discrete* environment with an action mapping and action space 
  depending on the step and delta specified. The action will be sum with current setpoint values instead of overwrite the latest action. 
  Then, the action is the current setpoints values with the increase instead of the index of the action_mapping for the continuous 
  environment in the next layer.

In [11]:
env=gym.make('Eplus-5Zone-hot-continuous-v1')
print('ORIGINAL ACTION SPACE: ',env.action_space)
env = DiscreteIncrementalWrapper(
        env, delta_temp=2, step_temp=0.5)
print('WRAPPED ACTION SPACE: ',env.action_space)
print('WRAPPED ACTION MAPPING: ',env.action_mapping)

[2023-06-28 14:26:21,857] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:21,859] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-06-28 14:26:21,860] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating building model OutPut:Variable and variables XML tree model for BVCTB connection.
[2023-06-28 14:26:21,861] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Setting up extra configuration in building model if exists...
[2023-06-28 14:26:21,861] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Setting up action definition in building model if exists...
ORIGINAL ACTION SPACE:  Box(-1.0, 1.0, (2,), float32)
New incremental action mapping: 17
{0: [0.0, 0.0], 1: [0.5, 0.0], 2: [1.0, 0.0], 3: [1.5, 0.0], 4: [2.0, 0.0], 5: [-0.5, 0.0], 6: [-1.0, 0.0], 7: [-1.5, 0.0], 8

maximum and minimum values are specified so that the setpoint increments and decrements do not go outside the agreed limits.
The delta and step values are used to determine how the discrete space of these increments and decrements will be constructed.
An example of how it works is shown below:

In [12]:
env.reset()
print('CURRENT SETPOINTS VALUES: ', env.current_setpoints)
for i in range(5):
    action=env.action_space.sample()
    _,_,_,_,info=env.step(action)
    print('Action number ',i,': ',env.action_mapping[action])
    print('Setpoints update: ', info['action'])
env.close()

[2023-06-28 14:26:21,870] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-06-28 14:26:22,015] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-continuous-v1-res3/Eplus-env-sub_run1


/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


CURRENT SETPOINTS VALUES:  [21, 25]
Action number  0 :  [-0.5, 0.0]
Setpoints update:  [20.5, 25.0]
Action number  1 :  [0.0, -1.0]
Setpoints update:  [20.5, 24.0]
Action number  2 :  [0.0, -1.5]
Setpoints update:  [20.5, 22.5]
Action number  3 :  [-2.0, 0.0]
Setpoints update:  [18.5, 22.5]
Action number  4 :  [-0.5, 0.0]
Setpoints update:  [18.0, 22.5]


/usr/local/lib/python3.10/dist-packages/gymnasium/spaces/box.py:230: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


[2023-06-28 14:26:27,653] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 


## Normalization Wrapper

This wrapper is used to transform observation received from simulator in values between 0 and 1.
It is possible to define a list of variables you want to normalize, if you don't define this list, all environment
variables will be included. 

In order to normalize values, it is necessary a dictionary in order to store max and min values. 
Then, if a environment variable is not included in the dictionary specified in wrapper constructor, then the normalization 
for that variable will be skipped.

In [13]:
#Original env
env=gym.make('Eplus-5Zone-hot-discrete-v1')
env = NormalizeObservation(
        env=env,
        ranges=RANGES_5ZONE,
        variables=[
            'Zone Thermostat Heating Setpoint Temperature(SPACE1-1)',
            'Zone Thermostat Cooling Setpoint Temperature(SPACE1-1)',
            'Zone Air Temperature(SPACE1-1)'])

[2023-06-28 14:26:27,735] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:27,735] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:27,735] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:27,735] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:27,735] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:27,737] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-06-28 14:26:27,737] EPLUS_ENV_5Zone-hot-discrete-v1_MainThre

We can see in the next code how the variables specified have been normalized correctly:

In [14]:
env.reset()
obs,_,_,_,_=env.step(env.action_space.sample())
obs_dict=dict(zip(env.variables['observation'],obs))
env.close()
print('OBSERVATION WITH NORMALIZATION IN VARIABLES SELECTED: ',obs_dict)

[2023-06-28 14:26:27,752] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-06-28 14:26:27,752] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-06-28 14:26:27,752] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-06-28 14:26:27,752] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-06-28 14:26:27,752] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-06-28 14:26:27,900] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-discrete-v1-res5/Eplus-env-sub_run1
[2023-06-28 14:26:27,900] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-discrete-v1

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[2023-06-28 14:26:33,646] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-06-28 14:26:33,646] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-06-28 14:26:33,646] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-06-28 14:26:33,646] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-06-28 14:26:33,646] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
OBSERVATION WITH NORMALIZATION IN VARIABLES SELECTED:  {'year': 1991.0, 'month': 1.0, 'day': 1.0, 'hour': 0.0, 'Site Outdoor Air Drybulb Temperature(Environment)': 4.4, 'Site Outdoor Air Relative Humidity(Environment)': 65.0, 'Site Wind Speed(Environment)': 3.875, 'Site Wind Direction(Environment)': 145.0, 'Site Diffuse Solar Radiation Rate per Area(Environment)': 0.0, 'Site Direct 

## Logger Wrapper

Wrapper for logging all interactions between agent and environment. Logger class can be selected
in the constructor if other type of logging is required. For more information about *Sinergym* Logger visit [Logger](https://ugr-sail.github.io/sinergym/compilation/main/pages/output.html#logger)

In [15]:
env=gym.make('Eplus-5Zone-hot-discrete-v1')
env=LoggerWrapper(env)


env.reset()    
terminated = False
current_month = 0
while not terminated:
    a = env.action_space.sample()
    _,_,terminated,_,_=env.step(a)
env.close()

[2023-06-28 14:26:33,719] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:33,719] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:33,719] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:33,719] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:33,719] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:33,719] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:33,721] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating 

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


[2023-06-28 14:26:43,474] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-06-28 14:26:43,474] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-06-28 14:26:43,474] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-06-28 14:26:43,474] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-06-28 14:26:43,474] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 
[2023-06-28 14:26:43,474] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus simulation closed successfully. 


Now, you can see in Sinergym output folder that you will have available `progress.csv` file and `monitor.csv` files in each episode.

## Multi Observation Wrapper

This wrapper will stack observation received in a history queue (size can be customized).

In [16]:
#Original environment
env=gym.make('Eplus-5Zone-hot-discrete-v1')
obs, info=env.reset()
print('BEFORE MULTI OBSERVATION: ',obs)

#Multi Observation environment
env=MultiObsWrapper(env, n=5, flatten=True)
obs, info=env.reset()

[2023-06-28 14:26:43,528] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:43,528] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:43,528] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:43,528] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:43,528] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:43,528] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:43,528] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating 

[2023-06-28 14:26:43,529] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-06-28 14:26:43,529] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-06-28 14:26:43,529] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-06-28 14:26:43,529] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-06-28 14:26:43,529] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-06-28 14:26:43,532] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:Updating building model OutPut:Variable and variabl

/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(


BEFORE MULTI OBSERVATION:  [1.9910000e+03 1.0000000e+00 1.0000000e+00 0.0000000e+00 4.0000000e+00
 6.9000000e+01 3.0999999e+00 1.3000000e+02 0.0000000e+00 0.0000000e+00
 2.1000000e+01 2.5000000e+01 2.0920815e+01 2.3152283e+01 0.0000000e+00
 0.0000000e+00 5.9761237e+02]
[2023-06-28 14:26:48,851] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-06-28 14:26:48,851] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-06-28 14:26:48,851] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-06-28 14:26:48,851] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-06-28 14:26:48,851] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed successfully. 
[2023-06-28 14:26:48,851] EPLUS_ENV_5Zone-hot-discrete-v1_MainThread_ROOT INFO:EnergyPlus episode completed success

In [17]:
print('AFTER MULTI OBSERVATION: ',obs)
env.close()

AFTER MULTI OBSERVATION:  [1.9910000e+03 1.0000000e+00 1.0000000e+00 0.0000000e+00 4.0000000e+00
 6.9000000e+01 3.0999999e+00 1.3000000e+02 0.0000000e+00 0.0000000e+00
 2.1000000e+01 2.5000000e+01 2.0920815e+01 2.3152283e+01 0.0000000e+00
 0.0000000e+00 5.9761237e+02 1.9910000e+03 1.0000000e+00 1.0000000e+00
 0.0000000e+00 4.0000000e+00 6.9000000e+01 3.0999999e+00 1.3000000e+02
 0.0000000e+00 0.0000000e+00 2.1000000e+01 2.5000000e+01 2.0920815e+01
 2.3152283e+01 0.0000000e+00 0.0000000e+00 5.9761237e+02 1.9910000e+03
 1.0000000e+00 1.0000000e+00 0.0000000e+00 4.0000000e+00 6.9000000e+01
 3.0999999e+00 1.3000000e+02 0.0000000e+00 0.0000000e+00 2.1000000e+01
 2.5000000e+01 2.0920815e+01 2.3152283e+01 0.0000000e+00 0.0000000e+00
 5.9761237e+02 1.9910000e+03 1.0000000e+00 1.0000000e+00 0.0000000e+00
 4.0000000e+00 6.9000000e+01 3.0999999e+00 1.3000000e+02 0.0000000e+00
 0.0000000e+00 2.1000000e+01 2.5000000e+01 2.0920815e+01 2.3152283e+01
 0.0000000e+00 0.0000000e+00 5.9761237e+02 1.991000

## Nesting wrappers

All the wrappers listed in this notebook can be nested in order to combine functionalities added. However, **the order** in which these wrappers are nested is critical to their proper functionality. We strongly recommend that it be in the same order in which they appear here or in the documentation. In case you want to nest a subset of the total, it is not a problem as long as you continue respecting this order.
We see and example:

In [18]:
env = gym.make('Eplus-5Zone-hot-continuous-v1')
env = MultiObjectiveReward(
    env=env,
    reward_terms=[
        'reward_energy',
        'reward_comfort'])
env = PreviousObservationWrapper(env, previous_variables=[
    'Zone Thermostat Heating Setpoint Temperature(SPACE1-1)',
    'Zone Thermostat Cooling Setpoint Temperature(SPACE1-1)',
    'Zone Air Temperature(SPACE1-1)'])
env = DatetimeWrapper(env)
env = DiscreteIncrementalWrapper(
    env, delta_temp=2, step_temp=0.5)
env = NormalizeObservation(
    env=env,
    ranges=RANGES_5ZONE,
    variables=[
        'Zone Thermostat Heating Setpoint Temperature(SPACE1-1)',
        'Zone Thermostat Cooling Setpoint Temperature(SPACE1-1)',
        'Zone Air Temperature(SPACE1-1)',
        'Zone Thermostat Heating Setpoint Temperature(SPACE1-1)_previous',
        'Zone Thermostat Cooling Setpoint Temperature(SPACE1-1)_previous',
        'Zone Air Temperature(SPACE1-1)_previous'])
env = LoggerWrapper(env=env, flag=True)
env = MultiObsWrapper(env=env, n=5, flatten=True)

[2023-06-28 14:26:54,249] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:54,249] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating Building model ExternalInterface object if it is not present...
[2023-06-28 14:26:54,251] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-06-28 14:26:54,251] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating Building model Site:Location and SizingPeriod:DesignDay(s) to weather and ddy file...
[2023-06-28 14:26:54,252] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating building model OutPut:Variable and variables XML tree model for BVCTB connection.
[2023-06-28 14:26:54,252] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Updating building model OutPut:Variable and variables XML tree model for BVCTB connecti

Now we just simply use the environment with the wrappers, for example:

In [19]:
for i in range(1):
    obs, info = env.reset()
    terminated = False
    current_month = 0
    while not terminated:
        a = env.action_space.sample()
        obs, reward, terminated, truncated, info = env.step(a)
        if info['month'] != current_month:  # display results every month
            current_month = info['month']
            print('Reward: ', reward, info)
env.close()

[2023-06-28 14:26:54,261] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-06-28 14:26:54,261] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:Creating new EnergyPlus simulation episode...
[2023-06-28 14:26:54,361] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-continuous-v1-res4/Eplus-env-sub_run1
[2023-06-28 14:26:54,361] EPLUS_ENV_5Zone-hot-continuous-v1_MainThread_ROOT INFO:EnergyPlus working directory is in /workspaces/sinergym/examples/Eplus-env-5Zone-hot-continuous-v1-res4/Eplus-env-sub_run1


/usr/local/lib/python3.10/dist-packages/opyplus/weather_data/weather_data.py:493: FutureWarning: the 'line_terminator'' keyword is deprecated, use 'lineterminator' instead.
  epw_content = self._headers_to_epw(use_datetimes=use_datetimes) + df.to_csv(
/usr/local/lib/python3.10/dist-packages/gymnasium/spaces/box.py:230: UserWarning: WARN: Casting input x to numpy array.
  gym.logger.warn("Casting input x to numpy array.")


Reward:  [-0.05934675774482191, -0.0] {'timestep': 1, 'time_elapsed': 900, 'year': 1991, 'month': 1, 'day': 1, 'hour': 0, 'action': [21.0, 24.5], 'reward': -0.029673378872410956, 'reward_energy': -0.05934675774482191, 'reward_comfort': -0.0, 'total_energy': 593.4675774482191, 'abs_comfort': 0.0, 'temperatures': [20.85245579021927]}
Reward:  [-0.011010506234462241, -0.0] {'timestep': 2976, 'time_elapsed': 2678400, 'year': 1991, 'month': 2, 'day': 1, 'hour': 0, 'action': [21.0, 29.5], 'reward': -0.0055052531172311205, 'reward_energy': -0.011010506234462241, 'reward_comfort': -0.0, 'total_energy': 110.1050623446224, 'abs_comfort': 0.0, 'temperatures': [21.57642594573431]}
Reward:  [-0.01626129654840774, -0.0] {'timestep': 5664, 'time_elapsed': 5097600, 'year': 1991, 'month': 3, 'day': 1, 'hour': 0, 'action': [22.5, 26.5], 'reward': -0.00813064827420387, 'reward_energy': -0.01626129654840774, 'reward_comfort': -0.0, 'total_energy': 162.6129654840774, 'abs_comfort': 0.0, 'temperatures': [22